<a href="https://colab.research.google.com/github/prabal5ghosh/UCA-M2-SEMESTER1/blob/main/deep%20learning/Copy_of_Lab8Chatbots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading up a pre-trained LLM

We are going to use the Hugging Face transformers library and repositories to load a small Large Language Model: Qwen2.5-7B-Instruct.

Qwen is a family of open-weight LLM hosted on the Hugging Face servers (along with many other open-weight LLMs like Gemma and LLama).
It is available in a number of versions of varying sizes: 7B, 14B and even bigger models.

We are going to directly use the version of Qwen2.5-7B fine-tuned on a instruct dataset to fit the behavior expected of a typical chatbot.

You can find other versions of Qwen (e.g. the non fine-tuned one) and other LLMs on the Hugging Face repositories. Keep in mind that your GPUs only have 24GB of RAM!


In [1]:
!pip install bitsandbytes
!pip install accelerate
!pip install transformers

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.9 MB/s eta 0:00:00


In [2]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

model_id = "Qwen/Qwen2.5-0.5B-Instruct"
model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        torch_dtype="auto")
        #quantization_config=quantization_config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

# Building a basic prompt

We are going to start by trying to build a basic prompt out of a simple sentence: "Hello, how can I help you?".

Use the tokenizer object to transform the sentence in individual tokens. Remember, we want to get a list of token ids corresponding to the atomic tokens extracted from the sentence.

In [7]:
input_text = "Hello, how can I help you?"
print(input_text)
input_tokens = tokenizer(input_text, return_tensors = 'pt')
print(input_tokens)
input_ids = input_tokens.input_ids.to("cuda")
print(input_ids)

Hello, how can I help you?
{'input_ids': tensor([[9707,   11, 1246,  646,  358, 1492,  498,   30]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}
tensor([[9707,   11, 1246,  646,  358, 1492,  498,   30]], device='cuda:0')


Now that we have our input, try to get the output logits of the network regarding the next word in the sentence.

In [8]:
output = model(input_ids)
next_token_logits = output.logits[0, -1, :]
print(next_token_logits)


# 0: Selects the first sequence in the batch.
# -1  is the sequence length

# :

tensor([ 4.5000,  3.0781,  5.3750,  ..., -4.8125, -4.8125, -4.8125],
       device='cuda:0', dtype=torch.bfloat16, grad_fn=<SliceBackward0>)


You can look at the most likely next token predicted by the model

In [11]:
next_token_id = torch.argmax(next_token_logits, dim = -1).unsqueeze(0)
print(next_token_id)

tensor([358], device='cuda:0')


And this token can be decoded into an actual output token!

In [14]:
answer_text = tokenizer.decode(next_token_id[0].cpu().tolist(), skip_special_tokens= False)
print(answer_text)

 I


# Determinist decoding algorithm

Fill in the inference code below to infer the most likely sequence of characters. You can look at what you wrote before to complete the initial prompt


In [ ]:
input_text = "Hello, how can I help you?"

input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

next_input = input_ids
max_length = 80
current_length = input_ids.shape[1]

while True:
    if current_length >= max_length:
        break


    output = None
    next_token_logits = None

    next_token_id = None
    next_token = None
    print(next_token, end='', flush=True)

    next_input = None

    current_length += 1

    if next_token_id[0].item() == tokenizer.eos_token_id:
        break


Well, the prediction is a bit strange for the output. Any idea why? Can you solve the issue? Keep in mind we are using a Instruct fine tuned model!

In [ ]:
input_text = None

input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

next_input = input_ids
max_length = 80
current_length = input_ids.shape[1]

while True:
    if current_length >= max_length:
        break


    output = None
    next_token_logits = None

    next_token_id = None
    next_token = None
    print(next_token, end='', flush=True)

    next_input = None

    current_length += 1

    if next_token_id[0].item() == tokenizer.eos_token_id:
        break


# More complex inference schemes

We do not have to stick to determinist predictions. Remember that the LLM outputs are basically probabilities of next words. Can you reframe the prediction problem to make it non-deterministic? You can use torch.multinomial().

In [ ]:
input_text = None

input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

next_input = input_ids
max_length = 100
current_length = input_ids.shape[1]

while True:
    if current_length >= max_length:
        break


    output = None
    next_token_logits = None
    next_token_id = None
    next_token = None
    print(next_token, end='', flush=True)

    next_input = None

    current_length += 1

    if next_token_id[0].item() == tokenizer.eos_token_id:
        break


You can try to play with other ideas for better sentences, change up the prompts!

Better, you try to implement prompt-engineering techniques to modify the behavior of the model to do what you want!

# Bonus: With manual word embeddings (e.g. for prompt tuning)

You can try to do away with the word tokens and directly work on the embeddings (for things like prompt-tuning). Can you rewrite your previous code to first extract the embeddings of the input sentence, and then feed these embeddings (possibly modified) into the model to generate the next word?

In [ ]:
import torch
import torch.nn.functional as F

input_text = None

input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

next_input = input_ids
max_length = 100  # Change this to your desired output length
current_length = input_ids.shape[1]
k = 10  # Number of tokens to sample from. Adjust as necessary. Greater k = greater variability.

while True:
    if current_length >= max_length:  # Check if we've reached the length limit
        break

    inputs_embeds = None
    output = None

    next_token_logits = None

    next_token_id = None
    next_token = None

    print(next_token, end='', flush=True)

    next_input = None

    current_length += 1

    if next_token_id[0].item() == tokenizer.eos_token_id:
        break